# DeepDream

DeepDream is an artistic image-modification technique that uses the representations learned by convolutional neural networks. It was first realeased by Google in the summer of 2015.

![](dd.jpg)

## Implementing deep dream in keras

We'll start from a convnet pretrained on ImageNet.

In [1]:
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


In [2]:
K.set_learning_phase(0) # Disable all training operations
model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


Next, we'll compute hte loss: the quantity we'll sekk to maximize using the gradient ascent process.

We will maximize the activation of all filters in anumber of layers. Specifically, we'll maximize a weighted sum of the L2 nrom of the activations of a set of high-level layers.

Lower layers result in geometric patterns, whearas higher layers result in visuals in which you can recognize some classes from imagenet (dogs, birds..)

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [4]:
layers_contrib = { # How much the layer'activation contributes to the loss
    'mixed2': .2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5
}

In [7]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layers_contrib:
    coeff = layers_contrib[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff * K.sum(K.square(activation[:, 2: - 2, 2: -2, :])) / scaling

Gradient ascent process

In [8]:
dream = model.input

grads = K.gradients(loss, dream)[0] # Gradients regards the loss
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) # Normalize the gradients 

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    return fetch_loss_and_grads([x])

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_val, grads_val = eval_loss_and_grads(x)
        if max_loss is not None and loss_val > max_loss:
            break
        print('...Loss value at', i, ':', loss_val)
        x += step * grads_val
    return x

Finally: The actual DeepDream algorithm. First we define a list of scales (also called *octaves*) at which to process the images. Each successive scale is larger than the previous one by factor 1.4: you start processing a small image an them increasingly scale it up

In [10]:
import scipy
import numpy as np
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname, pil_img)


def preprocess_image(image_path):
    # Util function to open, resize and format pictures
    # into appropriate tensors.
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    # Util function to convert a tensor into a valid image.
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [11]:
import numpy as no

step = .01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = 'original_dd.jpeg'

# Load the image into a Numpy array
img = preprocess_image(base_image_path)

# We prepare a list of shape tuples
# defining the different scales at which we will run gradient ascent
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

# Reverse list of shapes, so that they are in increasing order
successive_shapes = successive_shapes[::-1]

# Resize the Numpy array of the image to our smallest scale
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')

Processing image shape (169, 255)
...Loss value at 0 : 1.7125533
...Loss value at 1 : 2.0141518
...Loss value at 2 : 2.6712263
...Loss value at 3 : 3.2441173
...Loss value at 4 : 3.8333526
...Loss value at 5 : 4.3792906
...Loss value at 6 : 4.9755855
...Loss value at 7 : 5.5752254
...Loss value at 8 : 6.150064
...Loss value at 9 : 6.6743608
...Loss value at 10 : 7.224783
...Loss value at 11 : 7.776662
...Loss value at 12 : 8.391129
...Loss value at 13 : 9.030373
...Loss value at 14 : 9.568507


/home/guillem/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  app.launch_new_instance()


Processing image shape (237, 357)
...Loss value at 0 : 3.0113738
...Loss value at 1 : 4.495738
...Loss value at 2 : 6.194947
...Loss value at 3 : 8.150198
Processing image shape (332, 500)


/home/guillem/anaconda3/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


...Loss value at 0 : 3.5699153
...Loss value at 1 : 5.793063
...Loss value at 2 : 8.471212
